#Load data

In [ ]:
import zipfile
import os
with zipfile.ZipFile('NMNISTsmall.zip') as zip_file:
    for member in zip_file.namelist():
        if not os.path.exists('./' + member):
            zip_file.extract(member, './')

"\nimport zipfile\nimport os\nwith zipfile.ZipFile('NMNISTsmall.zip') as zip_file:\n    for member in zip_file.namelist():\n        if not os.path.exists('./' + member):\n            zip_file.extract(member, './')\n"

In [ ]:
import numpy as np
import spikeFileIO as io
from matplotlib import pyplot as plt
import os

from matplotlib.gridspec import GridSpec
import seaborn as sns

import torch
import torch.nn as nn
import torchvision

In [ ]:
#lets collect addresses of training and test data
def load(fname):
    f = open(fname,'r')

    data = []
    for line in f.readlines():
        data.append(line.replace('\n','').split(' '))

    f.close()

    return data

path = 'NMNISTsmall/'
#train data
training_files = load('NMNISTsmall/train1K.txt')
training_files = training_files[1:]

training_addrs = []
training_labels = []
for i in range (len(training_files)):
  training_addrs.append(path+training_files[i][0][:-2]+'.bs2')
  training_labels.append(training_files[i][0][-1])

#test data
test_files = load('NMNISTsmall/test100.txt')
test_files = test_files[1:]

test_addrs = []
test_labels = []
for i in range (len(test_files)):
  test_addrs.append(path+test_files[i][0][:-2]+'.bs2')
  test_labels.append(test_files[i][0][-1])

"\n#lets collect addresses of training and test data\ndef load(fname):\n    f = open(fname,'r')\n\n    data = []\n    for line in f.readlines():\n        data.append(line.replace('\n','').split(' '))\n\n    f.close()\n\n    return data\n\npath = 'NMNISTsmall/'\n#train data\ntraining_files = load('NMNISTsmall/train1K.txt')\ntraining_files = training_files[1:]\n\ntraining_addrs = []\ntraining_labels = []\nfor i in range (len(training_files)):\n  training_addrs.append(path+training_files[i][0][:-2]+'.bs2')\n  training_labels.append(training_files[i][0][-1])\n\n#test data\ntest_files = load('NMNISTsmall/test100.txt')\ntest_files = test_files[1:]\n\ntest_addrs = []\ntest_labels = []\nfor i in range (len(test_files)):\n  test_addrs.append(path+test_files[i][0][:-2]+'.bs2')\n  test_labels.append(test_files[i][0][-1])\n"

Instead of retaining only the first earliest spikes, we can retain more.

In [ ]:
def xyp21dim_onlyfirstspikepreserved(TD):
  TD_temporal = torch.ones((34,34,2))*2.8 #assumed that 2.8 is a mask number, identified as no-spiking time. 2.8 is due to historical reason
  t_max = TD.t[-1]
  for i in range(TD.x.shape[0]):
    if TD_temporal[TD.x[i], TD.y[i], TD.p[i]] == 2.8:
      TD_temporal[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max #normalisation
  return TD_temporal

In [ ]:
def xyp21dim_fewmorespikes(TD):
  TD_temporal_1 = torch.ones((34,34,2))*2.8 #assumed that 2.8 is a mask number, identified as no-spiking time. 2.8 is due to historical reason
  TD_temporal_2 = torch.ones((34,34,2))*2.8
  TD_temporal_3 = torch.ones((34,34,2))*2.8
  TD_temporal_4 = torch.ones((34,34,2))*2.8
  TD_temporal_5 = torch.ones((34,34,2))*2.8
  TD_temporal_6 = torch.ones((34,34,2))*2.8

  pixel_occur_first_time = []
  pixel_occur_second_time = []
  pixel_occur_third_time = []
  pixel_occur_forth_time =[]
  pixel_occur_fifth_time =[]
  pixel_occur_sixth_time =[]

  t_max = TD.t[-1]

  for i in range(TD.x.shape[0]):
    if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_first_time:
      TD_temporal_1[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
      pixel_occur_first_time.append([TD.x[i], TD.y[i], TD.p[i]])
    else:
      if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_second_time:
        TD_temporal_2[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
        pixel_occur_second_time.append([TD.x[i], TD.y[i], TD.p[i]])
      else:
        if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_third_time:
          TD_temporal_3[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
          pixel_occur_third_time.append([TD.x[i], TD.y[i], TD.p[i]])
        else:
          if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_forth_time:
            TD_temporal_4[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
            pixel_occur_forth_time.append([TD.x[i], TD.y[i], TD.p[i]])
          else:
            if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_fifth_time:
              TD_temporal_5[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
              pixel_occur_fifth_time.append([TD.x[i], TD.y[i], TD.p[i]])
            else:
              if [TD.x[i], TD.y[i], TD.p[i]] not in pixel_occur_sixth_time:
                TD_temporal_6[TD.x[i], TD.y[i], TD.p[i]] = TD.t[i]/t_max
                pixel_occur_sixth_time.append([TD.x[i], TD.y[i], TD.p[i]])
  return TD_temporal_1, TD_temporal_2,TD_temporal_3, TD_temporal_4, TD_temporal_5, TD_temporal_6, pixel_occur_first_time, pixel_occur_second_time,pixel_occur_third_time,pixel_occur_forth_time, pixel_occur_fifth_time,pixel_occur_sixth_time

change the mode to have different feature vectors

In [ ]:
def dense_data_generator(X, y, batch_size, samplingTime=1, samplingLength=300, shuffle=True, device = 'cuda', dtype=torch.float, mode = 'concatenation6'):
    """ This generator takes training data's address and generates spiking network input as dense tensors. 

    Args:
        X: The data ( 'data/userx_lighting_conditions/y.npy' )
        y: The labels
        batch_size: batch size
        samplingTime: period of sampling; default is 1ms
        samplingLength: in SLAYER training as well as testing, only the first 1.5 s out of ≈ 6 s of action video for each clas
    """ 
    nTimeBins = int(samplingLength/samplingTime)
    labels_ = np.array(y,dtype=np.int)
    number_of_batches = len(X)//batch_size
    sample_index = np.arange(len(X))

    if shuffle:
        np.random.shuffle(sample_index)

    total_batch_count = 0
    counter = 0
    while counter<number_of_batches:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        #initialise X_batch
        if mode == 'normal': #the input signal is only the first spiking time
          X_batch = torch.empty((batch_size,2*34*34),dtype=dtype)
        elif mode == 'concatenation': #the input signal is the first spiking time concatenated with the second spiking time
          X_batch = torch.empty((batch_size,2*34*34*2),dtype=dtype)
        elif mode == 'concatenation6': #the input signal is the first spiking time concatenated with the second spiking time
          X_batch = torch.empty((batch_size,2*34*34*6),dtype=dtype)   
        #assign values to X_batch
        for bc,i in enumerate(batch_index):
            TD = io.read2Dspikes(X[i])
            if mode == 'normal':
              X_batch_temp = xyp21dim_onlyfirstspikepreserved(TD)
              #X_batch_temp,X_batch_temp_second,_,_ = xyp21dim_first2spikepreserved_test(TD)
              X_batch[bc] = X_batch_temp.view((-1))

            elif mode == 'concatenation':
              X_batch_temp_first,X_batch_temp_second,_,_ = xyp21dim_first2spikepreserved_test(TD)
              #X_batch_temp_second = downsampling4(X_batch_temp_second, mode = 'average')
              X_batch[bc] = torch.cat((X_batch_temp_first.view((-1)), X_batch_temp_second.view((-1))), 0)

            elif mode == 'concatenation6':
              TD_temporal_1, TD_temporal_2,TD_temporal_3, TD_temporal_4, TD_temporal_5, TD_temporal_6, _,_,_,_,_,_ = xyp21dim_fewmorespikes(TD)              #X_batch_temp_second = downsampling4(X_batch_temp_second, mode = 'average')
              X_batch[bc] = torch.cat((TD_temporal_1.view((-1)), TD_temporal_2.view((-1)),TD_temporal_3.view((-1)), TD_temporal_4.view((-1)),TD_temporal_5.view((-1)), TD_temporal_6.view((-1))), 0)


            #flatten dimensions and save in X_batch
        y_batch = torch.tensor(labels_[batch_index],device=device)
        yield [X_batch.to(device=device), y_batch.to(device=device)] 
        counter+=1    

# Kernel

Kernel generates spiking time of a neuron given some input spike time 

In [ ]:
def alpha_kernel_response_temporal_v2(input_time, weights, tau = 1, thre = 1): #faster implementation
  #argument: input_time shape = (n_batch x n_input);  weights shape = (n_input x n_output)
  #alpha_kernel_response is t*e^(tau*t); the increment of this kernel is (1+t*tau)*exp(tau*t)
  n_batch = input_time.shape[0]
  n_input = input_time.shape[1]
  n_output = weights.shape[1]
  precision = 100.0
  index = torch.where(input_time != 2.8)
  t_max = torch.max(input_time[index]) + 4.0 #4 is the designed kernel length
  n_max = precision*t_max

  v_accu = torch.zeros((n_batch, n_output), device = input_time.device)
  spike_time = torch.zeros_like(v_accu)
  spike_time_mem = torch.ones_like(v_accu)
  for t in range(int(n_max)):
    t_clamp = torch.clamp(t/precision -input_time, 0, t_max)
    invalid_incr = (t_clamp==0)
    exptau = torch.exp(-tau*t_clamp)
    v_incr = exptau*(1-tau*t_clamp)/precision + 0.5/(precision**2) *exptau*tau*(2+tau)  #unweighted dv/dt
    v_incr[invalid_incr] = 0
    v_incr = torch.mm(v_incr, weights) #dv/dt * delta_t
    v_accu += v_incr #update the membrane potential at time instance t
    index = (v_accu>thre) & (spike_time_mem == 1) #output index to update the earilest spiking time
    spike_time[index] = t/precision
    spike_time_mem[index] = 0 #mark the updated output index such that it will not be updated again
  
  index_nospike = (spike_time == 0)
  spike_time[index_nospike] = 2.8 #an arbirary number that identifies no-spiking neuron
  
  return spike_time.to(input_time.device)

# Network Configuratoin

Here we configure a SNN with input layer of a size 2x34x34, a hidden layer of a size 100, and an output layer of a size 10. Moreover, both the input layer and the hidden layer have one bias to prevent neuron from not spiking.

# Helper functions

helper function that solves W-Lambert function

In [ ]:
#https://github.com/google/ihmehimmeli/blob/master/tempcoding/lambertw.cc
def LambertW0InitialGuess_vec(x):
  kNearBranchCutoff = -0.3235
  kE = 2.718281828459045
  x_copy = x.clone()

  # Sqrt approximation near branch cutoff.
  index1 = (x < kNearBranchCutoff)
  x_copy[index1] = -1.0+torch.sqrt(2.0*(1+kE*x[index1]))

  # Taylor series between [-1/e and 1/e].
  index2 = (x > kNearBranchCutoff) & (x < -kNearBranchCutoff)
  x_copy[index2] = x[index2] * (1 + x[index2] * (-1 + x[index2] * (3.0 / 2.0 - 8.0 / 3.0 * x[index2])))

  #Series of piecewise linear approximation
  index3 = (x > -kNearBranchCutoff) & (x < 0.6)
  x_copy[index3] =  0.23675531078855933 + (x[index3] - 0.3) * 0.5493610866617109
  
  index4 = (x > 0.6) & (x < 0.8999999999999999)
  x_copy[index4] = 0.4015636367870726 + (x[index4] - 0.6) * 0.4275644294878729

  index5 = (x > 0.8999999999999999) & (x < 1.2)
  x_copy[index5] = 0.5298329656334344 + (x[index5] - 0.8999999999999999) * 0.3524368357714513

  index6 = (x > 1.2) & (x < 1.5)
  x_copy[index6] = 0.6355640163648698 + (x[index6] - 1.2) * 0.30099113800452154

  index7 = (x > 1.5) & (x < 1.8)
  x_copy[index7] = 0.7258613577662263 + (x[index7] - 1.5) * 0.2633490154764343
  
  index8 = (x > 1.8) & (x < 2.0999999999999996)
  x_copy[index8] = 0.8048660624091566 + (x[index8] - 1.8) * 0.2345089875713013;
  
  index9 = (x > 2.0999999999999996) & (x < 2.4)
  x_copy[index9] =  0.8752187586805469 + (x[index9] - 2.0999999999999996) * 0.2116494532726034

  index10 = (x > 2.4) & (x < 2.6999999999999997)
  x_copy[index10] = 0.938713594662328 + (x[index10] - 2.4) * 0.19305046534383152

  index11 = (x > 2.6999999999999997) & (x < 2.9999999999999996)
  x_copy[index11] = 0.9966287342654774 + (x[index11] - 2.6999999999999997) * 0.17760053566187495
  
  #asymptotic approxiamtion
  index12 = ~(index1 + index2 + index3 + index4 + index5 + index6 +index7 +index8 +index9 +index10 + index11)
  l = torch.log(x[index12])
  ll = torch.log(l)
  x_copy[index12] = l - ll + ll/l
  return x_copy

def LambertW0_vec(x):
  x_copy = x.clone()
  kReciprocalE = 0.36787944117
  kDesiredAbsoluteDifference = 1e-3
  kNumMaxIters = 10

  index1 =  (x < -kReciprocalE)
  x_copy[index1] = 0
  #return x, False

  index2 = (x == 0.0)
  x_copy[index2] = 0

  index3 = (x == -kReciprocalE)
  x_copy[index3] = -1 

  index4 = ~(index1 + index2 + index3)

  #current guess
  w_n = LambertW0InitialGuess_vec(x[index4])
  have_convergence = False

  #fritsch iteration
  for i in range(1):
    z_n = torch.log(x[index4] / w_n) - w_n
    q_n = 2.0 * (1.0 + w_n) * (1.0 + w_n + 2.0 / 3.0 * z_n)
    e_n = (z_n / (1.0 + w_n)) * ((q_n - z_n) / (q_n - 2.0 * z_n))
    w_n = w_n * (1.0 + e_n)
    #Done this way as the log is the expensive part above.
    #if (torch.abs(z_n) < kDesiredAbsoluteDifference):

  x_copy[index4] = w_n
  return x_copy
    


# Forward and backward propagation

In [ ]:
class SNN_process_v3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input, weight, bias_time, bias_weight):
        """
        Argument: input is input spike time of dimension (n_output)
                  weight is of dimension (n_input, n_output)
                  bias_time is of dimension (n_bias)
                  bias_weight is of dimension (n_bias, n_output)
        """
        #####firstly, we combine the bias and inputs, and their weights#####
        batch_size = input.shape[0]
        n_bias = bias_time.shape[0]
        bias_time_broadcasted = bias_time.view(1,n_bias).repeat(batch_size,1)
        x_n_bias = torch.cat((input, bias_time_broadcasted),-1)

        w_n_bias = torch.cat((weight,bias_weight),0)

        #####membrane dynamics begin here###################################
        tau =  2#0.181769
        theta = 0.5 #1.16732
        not_spike_time = 2.8 #if the neuron does not spike, the spike time is assigned to 2.8 sec. It is a potential problem
        nb_outputs = w_n_bias.shape[1]

        spk = alpha_kernel_response_temporal_v2(x_n_bias, w_n_bias, tau = tau, thre = theta)
        ctx.intermediate_results = x_n_bias, w_n_bias, spk, n_bias

        return spk

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        x_n_bias, w1_n_bias, spk, n_bias = ctx.intermediate_results
        tau = 2 #0.181769
        theta = 0.5 #1.16732
        input = x_n_bias
        batch_size = x_n_bias.shape[0]
        nb_inputs = w1_n_bias.shape[0]
        nb_outputs = w1_n_bias.shape[1]

        #t0 = time.time()
        #compute ingredients of gradients 
        AI = torch.zeros_like(spk).to(input.device)  #initialisation
        BI = torch.zeros_like(spk).to(input.device)
        t_out = torch.zeros_like(spk).to(input.device)
        WI = torch.zeros_like(spk).to(input.device)
        grad_input = torch.zeros((batch_size, nb_inputs, nb_outputs)).to(input.device)
        grad_weight = torch.zeros((batch_size, nb_inputs, nb_outputs)).to(input.device)

        #extend matrices to a common shape
        x_n_bias_modified = x_n_bias.unsqueeze(2).repeat(1,1,nb_outputs)
        grad_output_modified = grad_output.unsqueeze(1).repeat(1,nb_inputs,1)
        w1_n_bias_repeated = w1_n_bias.unsqueeze(0).repeat(batch_size,1,1)

        #find the index for valid inputs 
        spk_modified = spk.unsqueeze(1).repeat(1,nb_inputs,1)
        valid_index = x_n_bias_modified < spk_modified

        #print(valid_index)
        AI = torch.sum(torch.exp(x_n_bias_modified*tau)*w1_n_bias_repeated*valid_index, 1).to(x_n_bias.device)
        BI = torch.sum(torch.exp(x_n_bias_modified*tau)*x_n_bias_modified*w1_n_bias_repeated*valid_index, 1).to(x_n_bias.device)
        #t1 = time.time()
        #print('AI and BI', t1-t0)
        exploding_WI_idx_batch = []
        exploding_WI_idx_out = []

        WI_intermediate = -tau*theta/AI * torch.exp(tau*BI/AI)
        #print('WI_intermediate',WI_intermediate)
        valid_WI_index = WI_intermediate != float('inf')
        WI[valid_WI_index] = LambertW0_vec(WI_intermediate[valid_WI_index])
        spk[~valid_WI_index] = 2.8

        spk_modified = spk.unsqueeze(1).repeat(1,nb_inputs,1) #update spk_modified, as spk changes
       

        #extend matrices to a common shape
        AI_modified = AI.unsqueeze(1).repeat(1,nb_inputs,1)
        BI_modified = BI.unsqueeze(1).repeat(1,nb_inputs,1)
        WI_modified = WI.unsqueeze(1).repeat(1,nb_inputs,1)

        valid_weights = w1_n_bias_repeated*valid_index #only inputs contributed to the spike generation are recognised as valid inputs

        dtout_dtin = grad_output_modified*valid_weights*torch.exp(x_n_bias_modified)*(x_n_bias_modified - BI_modified/AI_modified + WI_modified + 1)/(AI_modified*(1+WI_modified))
        #no spk penalty
        no_spk_idx = torch.where(spk_modified == 2.8)
        dtout_dtin[no_spk_idx] = 0
        grad_input = torch.sum(dtout_dtin,2)

        dtout_dw = grad_output_modified*valid_index*torch.exp(x_n_bias_modified)*(x_n_bias_modified - BI_modified/AI_modified + WI_modified)/(AI_modified*(1+WI_modified))
        dtout_dw[no_spk_idx] = -1
        grad_weight = torch.sum(dtout_dw,0)

        ##clip at 100
        torch.clamp(grad_weight, min=-100, max=100)
        torch.clamp(grad_input, min=-100, max=100)

        #solve nan values
        index_nan_t = torch.where(grad_input != grad_input)
        grad_input[index_nan_t] = 0
        index_nan_w = torch.where(grad_weight != grad_weight)
        grad_weight[index_nan_w] = 0        
        
        index_nan_t = torch.where(grad_input == float('inf'))
        grad_input[index_nan_t] = 100
        index_nan_w = torch.where(grad_weight == float('inf'))
        grad_weight[index_nan_w] = 100  

        index_nan_t = torch.where(grad_input == float('-inf'))
        grad_input[index_nan_t] = -100
        index_nan_w = torch.where(grad_weight == float('-inf'))
        grad_weight[index_nan_w] = -100   

        return grad_input[:,:nb_inputs-n_bias], grad_weight[:nb_inputs-n_bias,:], grad_input[:,nb_inputs-n_bias:].view(batch_size,n_bias), grad_weight[nb_inputs-n_bias:,:].view(n_bias,nb_outputs) #I ignored the grad for the bias

snn_process_v3 = SNN_process_v3.apply

# Store the data in a tensor

In [ ]:
#load training data for training
import time
batch_size = 5
x_batch =[]
y_batch =[]
t0 = time.time()
for x_local, y_local in dense_data_generator(training_addrs, training_labels, batch_size, shuffle=True):
  x_batch.append(x_local)
  y_batch.append(y_local)
t1 = time.time()
print(t1-t0)

'\n#load training data for training\nimport time\nbatch_size = 5\nx_batch =[]\ny_batch =[]\nt0 = time.time()\nfor x_local, y_local in dense_data_generator(training_addrs, training_labels, batch_size, shuffle=True):\n  x_batch.append(x_local)\n  y_batch.append(y_local)\nt1 = time.time()\nprint(t1-t0)\n'

In [ ]:
#load trainig data for accuracy
for x_test, y_test in dense_data_generator(test_addrs, test_labels, len(test_addrs), shuffle=False):
  1 == 1
print(x_test.shape)
print(y_test.shape)

'\n#load trainig data for accuracy\nfor x_test, y_test in dense_data_generator(test_addrs, test_labels, len(test_addrs), shuffle=False):\n  1 == 1\nprint(x_test.shape)\nprint(y_test.shape)\n'

If the RAM is limited, we can store the data in the disk, and load from there

In [ ]:
'''
def training_data_generator_20(batch_number = 50, batch_size=20, device = 'cuda', dtype=torch.float):
    counter = 0
    while counter<batch_number:
        [x_batch, y_batch] = torch.load('drive/My Drive/digit_6spikes/digit_training_data_bs20_6spikes_{:02d}.pt'.format(counter))
        yield [x_batch[:,:2*34*34*2].to(device=device), y_batch.to(device=device)] 
        counter+=1

def test_data_generator_20(batch_number = 5, batch_size=20, device = 'cuda', dtype=torch.float):
    counter = 0
    while counter<batch_number:
        [x_batch, y_batch] = torch.load('drive/My Drive/digit_6spikes/digit_test_data_bs20_6spikes_{:02d}.pt'.format(counter))
        yield [x_batch[:,:2*34*34*2].to(device=device), y_batch.to(device=device)] 
        counter+=1
'''

# Classification accuracy

In [ ]:
#[w1,w2, bias1_weight, bias2_weight, bias1_time, bias2_time] = torch.load('params_baseline300.pt')

In [ ]:
def compute_classification_accuracy(x_data, y_data):
    """ Computes classification accuracy on supplied data in batches. """
    accs = []
    spk_hidden = snn_process_v3(x_data, torch.cat((w1,w1),0), bias1_time, bias1_weight)
    spk_out = snn_process_v3(spk_hidden, w2, bias2_time, bias2_weight)
    pred = torch.argmin(spk_out,1)
    acc = np.mean((y_data==pred).detach().cpu().numpy())
    accs.append(acc)
    return np.mean(accs)

# Training

In [ ]:
nb_inputs  = 2*34*34*2
nb_hidden  = 300
nb_outputs = 10
#we assume there is only 1 batch now

#weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

dtype=torch.float
if torch.cuda.is_available():
    device = torch.device("cuda")     
else:
    device = torch.device("cpu")

#initialisation
weight_scale = np.sqrt(2)

nb_bias1 = 10
nb_bias2 = 10


w1 = torch.empty((int(nb_inputs/2), nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=-0.275419*weight_scale/np.sqrt(nb_inputs+nb_hidden+nb_bias1), std=weight_scale/np.sqrt(nb_inputs+nb_hidden+nb_bias1)) #the mean is brought up to 1.5 to force neurons to spike initially

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=-0.275419*weight_scale/np.sqrt(nb_inputs+nb_hidden+nb_bias2), std=weight_scale/np.sqrt(nb_hidden+nb_outputs+nb_bias2))

bias1_time = torch.empty((nb_bias1),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.uniform_(bias1_time, a=0.0, b=1.0)
bias1_weight = torch.empty((nb_bias1, nb_hidden), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(bias1_weight, mean=7.83912*weight_scale/np.sqrt(nb_inputs+nb_bias1+nb_hidden), std=weight_scale/np.sqrt(nb_inputs+nb_bias1+nb_hidden))
#torch.nn.init.uniform_(bias1_weight, a=1.0, b=1.0)


bias2_time = torch.empty((nb_bias2),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.uniform_(bias2_time, a=0.0, b=1.0)
bias2_weight = torch.empty((nb_bias2, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(bias2_weight, mean=7.83912*weight_scale/np.sqrt(nb_outputs+nb_bias2+nb_hidden), std=weight_scale/np.sqrt(nb_outputs+nb_bias2+nb_hidden))
#torch.nn.init.uniform_(bias2_weight, a=1.0, b=1.0)



tensor([[0.5783, 0.5680, 0.5957, 0.6046, 0.5714, 0.6079, 0.6341, 0.6473, 0.7289,
         0.6496],
        [0.6293, 0.6639, 0.5421, 0.6772, 0.5713, 0.7271, 0.5755, 0.7280, 0.4283,
         0.6720],
        [0.5116, 0.6253, 0.5048, 0.5881, 0.6289, 0.7419, 0.6511, 0.6643, 0.5604,
         0.6214],
        [0.4492, 0.6639, 0.7306, 0.5801, 0.7676, 0.5942, 0.5643, 0.6592, 0.6418,
         0.6190],
        [0.7869, 0.5545, 0.5983, 0.6805, 0.5832, 0.5907, 0.5662, 0.4872, 0.7432,
         0.6460],
        [0.7115, 0.6319, 0.4918, 0.6156, 0.6969, 0.6360, 0.7009, 0.6715, 0.6126,
         0.5908],
        [0.6045, 0.6240, 0.7109, 0.5788, 0.5819, 0.6132, 0.4698, 0.6295, 0.6362,
         0.6480],
        [0.6074, 0.5554, 0.6071, 0.5647, 0.5152, 0.6309, 0.5130, 0.5369, 0.6650,
         0.5872],
        [0.6673, 0.5741, 0.5732, 0.6222, 0.5313, 0.6970, 0.5635, 0.5945, 0.6246,
         0.5745],
        [0.7320, 0.6229, 0.7566, 0.6198, 0.6161, 0.6717, 0.7276, 0.5397, 0.5090,
         0.5889]], device='c

In [ ]:
#[w1,w12, w2, bias1_weight, bias2_weight, bias1_time, bias2_time] = torch.load('params_baseline300.pt')

In [ ]:
import time
paramw = [w1, w12]
params = [ w2, bias1_weight, bias2_weight]
params_time = [bias1_time, bias2_time]

loss_fn = nn.CrossEntropyLoss()
m = nn.L1Loss()

#n_batch = int(1000/batch_size)
lr = 2e-3
lr_pulse = 6e-2

loss_list = []
loss_hist = []
train_accuracies = []
test_accuracies = []
train_accuracy= 0
pred_hist = torch.zeros(1000,3,2)

for e in range(100):
    optimizerw = torch.optim.Adam(paramw, lr=lr, betas=(0.9,0.999))
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    optimizer_bias_time = torch.optim.Adam(params_time, lr=lr_pulse, betas=(0.9,0.999))

    #####run snn begins###
    t0 = time.time()
    #for x_local, y_local in dense_data_generator(training_addrs, training_labels, batch_size, shuffle=False):
    for x_local, y_local in training_data_generator_20(batch_number = 50): #here I load the data from the disk
      spk_hidden = snn_process_v3(x_local, torch.cat((w1,w1),0), bias1_time, bias1_weight) #we concatenate two w1s to construct a new w1
      spk_out = snn_process_v3(spk_hidden, w2, bias2_time, bias2_weight)
      #####run snn ends#####

      loss = loss_fn(-spk_out, y_local)
      loss_list.append(loss.item())

      optimizerw.zero_grad()
      optimizer.zero_grad()
      optimizer_bias_time.zero_grad()

      loss.backward()

      optimizerw.step()
      optimizer.step()
      optimizer_bias_time.step()

    loss_list_t=torch.FloatTensor(loss_list) 
    loss_hist.append(torch.mean(loss_list_t))

    #compute accuracies
    acc_list = []
    for x_local, y_local in training_data_generator_20(batch_number = 50):
      acc = compute_classification_accuracy(x_local,y_local)
      acc_list.append(acc)
    train_accuracy = np.mean(acc_list)
    train_accuracies.append(train_accuracy)

    acc_list = []
    for x_local, y_local in test_data_generator_20(batch_number = 5):
        acc = compute_classification_accuracy(x_local,y_local)
        acc_list.append(acc)
    test_accuracy = np.mean(acc_list)
    test_accuracies.append(test_accuracy)


    t1 = time.time()
    print('epoch number: ', e, 'loss is ', loss_hist[e].item(), 'train accu is', train_accuracy, 'test accu is', test_accuracy, 'lr', lr, 'time=', t1-t0)


save all the weights

In [ ]:
torch.save([w1 ,w12,w2, bias1_weight, bias2_weight, bias1_time, bias2_time], 'params_baseline300.pt')

plot the learning curve

In [ ]:
plt.plot (train_accuracies,  label='train')
plt.plot (test_accuracies, label='test')
plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title('Accuracy')
#plt.savefig('accuracy.eps')

plot the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cf_total = np.zeros((10,10))
for i in range (10):
  spk_hidden = snn_process_v3(x_batch[i], w1, bias1_time, bias1_weight)
  spk_out = snn_process_v3(spk_hidden, w2, bias2_time, bias2_weight)
  y_pred = torch.argmin(spk_out,1)
  cf = confusion_matrix(y_batch[i].detach().cpu().numpy(), y_pred.detach().cpu().numpy())
  cf_total += cf
print(cf_total)

In [ ]:
cf_total_row = np.sum(cf_total, 1)
cf_total_row_expanded = np.zeros((10,10))
for i in range(10):
  cf_total_row_expanded[:,i] = cf_total_row
cf_norm = cf_total/cf_total_row_expanded
cf_norm = (100*cf_norm).astype(int)


In [ ]:

import seaborn as sn

sn.heatmap(cf_norm, annot=True)
plt.ylabel('true label')
plt.xlabel('predicted label')
plt.savefig('snn_baseline_cf_300tmax_v2')